In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier


In [2]:
train_data=pd.read_csv("Titanic Forecast/data/train.csv")
# test_data=train_data['Survived']
test_data=pd.read_csv("Titanic Forecast/data/test.csv")

In [3]:
datasets=[train_data,test_data]

In [4]:
clf=DecisionTreeClassifier()

In [5]:
train_data[['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [6]:
#checking traindata is equal to the total size of datas or not

In [7]:
for dataset in datasets:
    np.unique(dataset['PassengerId'].values).size,np.unique(dataset['PassengerId'].values).size == dataset.shape[0]

In [8]:
import re
regular = re.compile('\s(\S+\.)')
designation_train = np.array([re.search(regular,i)[1] for i in train_data['Name'].values])
designation_test = np.array([re.search(regular,i)[1] for i in test_data['Name'].values])


In [9]:
train_data=train_data.drop('Name',axis=1)
test_data=test_data.drop('Name',axis=1)

In [10]:
Title=pd.DataFrame({"Title":designation_train})


In [11]:
train_data = pd.concat([train_data, Title], axis=1)
Title=pd.DataFrame({"Title":designation_test})
test_data = pd.concat([test_data, Title], axis=1)

In [12]:
train_data['Title'].groupby(train_data['Title']).size()
test_data['Title'].groupby(test_data['Title']).size()

Title
Col.         2
Dona.        1
Dr.          1
Master.     21
Miss.       78
Mr.        240
Mrs.        72
Ms.          1
Rev.         2
Name: Title, dtype: int64

In [13]:
train_data['Title'] = train_data['Title'].replace('Mlle.','Miss.')
train_data['Title'] = train_data['Title'].replace('Ms.','Miss.')  
train_data['Title'] = train_data['Title'].replace('Mme.','Mrs.')
train_data['Title'] = train_data['Title'].replace(['Capt.','Col.','Major.'],'Army.')
train_data['Title'] = train_data['Title'].replace(['Countess.','Don.','Jonkheer.','Lady.','Sir.'],'Noble.')
test_data['Title'] = test_data['Title'].replace('Mlle.','Miss.')
test_data['Title'] = test_data['Title'].replace('Ms.','Miss.')  
test_data['Title'] = test_data['Title'].replace('Mme.','Mrs.')
test_data['Title'] = test_data['Title'].replace(['Capt.','Col.','Major.'],'Army.')
test_data['Title'] = test_data['Title'].replace(['Countess.','Don.','Jonkheer.','Lady.','Sir.'],'Noble.')

In [14]:
train_data['Title'].groupby(train_data['Title']).size()

Title
Army.        5
Dr.          7
Master.     40
Miss.      185
Mr.        517
Mrs.       126
Noble.       5
Rev.         6
Name: Title, dtype: int64

In [15]:
print('Number of survival:\n\n{}'.format(train_data[['Title','Survived']].groupby(train_data['Title']).mean()))


Number of survival:

         Survived
Title            
Army.    0.400000
Dr.      0.428571
Master.  0.575000
Miss.    0.702703
Mr.      0.156673
Mrs.     0.793651
Noble.   0.600000
Rev.     0.000000


In [16]:
train_data.drop('Ticket',axis=1,inplace=True)
test_data.drop('Ticket',axis=1,inplace=True)

In [17]:
train_data.set_index(['PassengerId'],inplace=True)
test_data.set_index(['PassengerId'],inplace=True)

In [18]:
def modify_cabin(cabin):
    if pd.isnull(cabin):
        cat = 'None'
    else:
        cat = cabin[0]
    return cat

In [19]:
cabin_category = np.array([modify_cabin(cab) for cab in train_data['Cabin'].values])
cabin_category_test = np.array([modify_cabin(cab) for cab in test_data['Cabin'].values])

In [20]:
train_data=train_data.assign(Cabs_Category=cabin_category)
test_data=test_data.assign(Cabs_Category=cabin_category_test)


In [21]:
train_data=train_data.drop('Cabin',axis=1)
test_data=test_data.drop('Cabin',axis=1)


In [22]:
print("no. of passengers{}".format(train_data['Cabs_Category'].groupby(train_data['Cabs_Category']).size()))

no. of passengersCabs_Category
A        15
B        47
C        59
D        33
E        32
F        13
G         4
None    687
T         1
Name: Cabs_Category, dtype: int64


In [23]:
survived_data = train_data.loc[train_data['Survived']==1,:]

In [24]:
died_data=train_data.loc[train_data['Survived']==0,:]

In [25]:
survived_male_data=survived_data.loc[survived_data['Sex']=='male']

In [26]:
died_male_data=died_data.loc[died_data['Sex']=='male']

In [27]:
survived_female_data=survived_data.loc[survived_data['Sex']=='female']

In [28]:
died_female_data=died_data.loc[died_data['Sex']=='female']

In [29]:
survived_male_data.groupby(survived_male_data['Sex']).size()

Sex
male    109
dtype: int64

In [30]:
survived_female_data.groupby(survived_female_data['Sex']).size()

Sex
female    233
dtype: int64

In [31]:
died_male_data.groupby(died_male_data['Sex']).size()

Sex
male    468
dtype: int64

In [32]:
died_female_data.groupby(died_female_data['Sex']).size()

Sex
female    81
dtype: int64

In [33]:
train_data["Age"].fillna(train_data.groupby("Title")["Age"].transform("median"), inplace=True)
test_data["Age"].fillna(test_data.groupby("Title")["Age"].transform("median"), inplace=True)

In [34]:
train_data["Age"].fillna(train_data.groupby("Title")["Age"].transform("median"), inplace=True)

In [35]:
test_data["Age"].fillna(test_data.groupby("Title")["Age"].transform("median"), inplace=True)

In [36]:
train_data[['Sex','Embarked','Title','Cabs_Category']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 4 columns):
Sex              891 non-null object
Embarked         889 non-null object
Title            891 non-null object
Cabs_Category    891 non-null object
dtypes: object(4)
memory usage: 74.8+ KB


In [37]:
train_data.loc[pd.isnull(train_data['Embarked'])]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Cabs_Category
PassengerId,,,,,,,,,,
62,1,1,female,38.0,0,0,80.0,NaN,Miss.,B
830,1,1,female,62.0,0,0,80.0,NaN,Mrs.,B


In [38]:
test_data.loc[pd.isnull(test_data['Embarked'])]

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Cabs_Category
PassengerId,,,,,,,,,


In [39]:
from sklearn.preprocessing import LabelEncoder
label_sex = LabelEncoder()
sex_numerical = label_sex.fit_transform(train_data['Sex'])
sex_numerical_classes = label_sex.classes_

label_sex_test = LabelEncoder()
sex_test_numerical = label_sex_test.fit_transform(test_data['Sex'])
sex_test_numerical_classes = label_sex_test.classes_
print('Classes of Sex feature:\n{}\n{}'.format(np.arange(len(sex_numerical_classes)),sex_numerical_classes))

Classes of Sex feature:
[0 1]
['female' 'male']


In [40]:
label_title = LabelEncoder()
title_numerical = label_title.fit_transform(train_data['Title'])
title_numerical_classes = label_title.classes_

label_title_test = LabelEncoder()
title_test_numerical = label_title_test.fit_transform(test_data['Title'])
title_test_numerical_classes = label_title_test.classes_
print('Classes of title feature:\n{}\n{}'.format(np.arange(len(title_numerical_classes)),title_numerical_classes))

Classes of title feature:
[0 1 2 3 4 5 6 7]
['Army.' 'Dr.' 'Master.' 'Miss.' 'Mr.' 'Mrs.' 'Noble.' 'Rev.']


In [41]:
label_Cabin = LabelEncoder()
Cabin_numerical = label_Cabin.fit_transform(train_data['Cabs_Category'])
Cabin_numerical_classes = label_Cabin.classes_

label_Cabin_test = LabelEncoder()
Cabin_test_numerical = label_Cabin_test.fit_transform(test_data['Cabs_Category'])
Cabin_test_numerical_classes = label_Cabin_test.classes_
print('Classes of title feature:\n{}\n{}'.format(np.arange(len(Cabin_numerical_classes)),Cabin_numerical_classes))

Classes of title feature:
[0 1 2 3 4 5 6 7 8]
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'None' 'T']


In [42]:
from sklearn.preprocessing import OneHotEncoder

In [43]:
encoder_Sex=OneHotEncoder(sparse=False)#will return array
sex_onehot = encoder_Sex.fit_transform(sex_numerical.reshape(-1,1))

encoder_Sex_test=OneHotEncoder(sparse=False)#will return array
sex_test_onehot = encoder_Sex_test.fit_transform(sex_test_numerical.reshape(-1,1))
print(sex_onehot)

print(sex_onehot.shape)

[[ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 ..., 
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]]
(891, 2)


In [44]:
encoder_Title=OneHotEncoder(sparse=False)#will return array
Title_onehot = encoder_Title.fit_transform(title_numerical.reshape(-1,1))
encoder_Title_test=OneHotEncoder(sparse=False)#will return array
Title_test_onehot = encoder_Title_test.fit_transform(title_test_numerical.reshape(-1,1))
print(Title_onehot)
print(Title_onehot.shape)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
(891, 8)


In [45]:
encoder_Cabin=OneHotEncoder(sparse=False)
Cabin_onehot=encoder_Cabin.fit_transform(Cabin_numerical.reshape(-1,1))

encoder_Cabin_test=OneHotEncoder(sparse=False)
Cabin_test_onehot=encoder_Cabin_test.fit_transform(Cabin_test_numerical.reshape(-1,1))
print(Cabin_onehot)
print(Cabin_onehot.shape)

[[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
(891, 9)


In [46]:
def oneHotLabels(train_data,column,onehot_labeled,class_labels):
    new_assigns={}
    for c_id,label in enumerate(class_labels):
        new_assigns[column+'_'+label]=onehot_labeled[:,c_id]
    train_data=train_data.assign(**new_assigns)
    return train_data

In [47]:
train_data=oneHotLabels(train_data,'Sex',sex_onehot,sex_numerical_classes)
train_data=train_data.drop('Sex',axis=1)

test_data=oneHotLabels(test_data,'Sex',sex_test_onehot,sex_test_numerical_classes)
test_data=test_data.drop('Sex',axis=1)

In [48]:
train_data=oneHotLabels(train_data,'Title',Title_onehot,title_numerical_classes)
train_data=train_data.drop('Title',axis=1)

test_data=oneHotLabels(test_data,'Title',Title_test_onehot,title_test_numerical_classes)
test_data=test_data.drop('Title',axis=1)

In [49]:
train_data=oneHotLabels(train_data,'Cabs_Category',Cabin_onehot,Cabin_numerical_classes)
train_data=train_data.drop('Cabs_Category',axis=1)

test_data=oneHotLabels(test_data,'Cabs_Category',Cabin_test_onehot,Cabin_test_numerical_classes)
test_data=test_data.drop('Cabs_Category',axis=1)

In [50]:
train_data['Fare'] = train_data['Fare'].fillna(train_data['Fare'].median())
test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].median())

In [51]:
train_data[['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked',
       'Sex_female', 'Sex_male', 'Title_Army.', 'Title_Dr.', 'Title_Master.',
       'Title_Miss.', 'Title_Mr.', 'Title_Mrs.', 'Title_Noble.', 'Title_Rev.',
       'Cabs_Category_A', 'Cabs_Category_B', 'Cabs_Category_C',
       'Cabs_Category_D', 'Cabs_Category_E', 'Cabs_Category_F',
       'Cabs_Category_G', 'Cabs_Category_None', 'Cabs_Category_T']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 26 columns):
Survived              891 non-null int64
Pclass                891 non-null int64
Age                   891 non-null float64
SibSp                 891 non-null int64
Parch                 891 non-null int64
Fare                  891 non-null float64
Embarked              889 non-null object
Sex_female            891 non-null float64
Sex_male              891 non-null float64
Title_Army.           891 non-null float64
Title_Dr.             891 non-null float64
Title_Master.         891 non-null float64
Title_Miss.           891 non-null float64
Title_Mr.             891 non-null float64
Title_Mrs.            891 non-null float64
Title_Noble.          891 non-null float64
Title_Rev.            891 non-null float64
Cabs_Category_A       891 non-null float64
Cabs_Category_B       891 non-null float64
Cabs_Category_C       891 non-null float64
Cabs_Category_D       891 non-null float64
C

In [52]:
train_data['Embarked'].value_counts().max()

644

In [53]:
train_data['Embarked'].value_counts().idxmax()

'S'

In [54]:
train_data['Embarked'] = train_data['Embarked'].fillna(train_data['Embarked'].value_counts().idxmax())
test_data['Embarked'] = test_data['Embarked'].fillna(test_data['Embarked'].value_counts().idxmax())

In [55]:
train_data['Embarked']=train_data['Embarked'].map({'S':0,'C':1,'Q':2}).astype(int)
test_data['Embarked']=test_data['Embarked'].map({'S':0,'C':1,'Q':2}).astype(int)

In [56]:
train_data.loc[train_data['Fare']<=7.91,'Fare']=0
train_data.loc[(train_data['Fare'] > 7.91) & (train_data['Fare'] <= 14.454), 'Fare'] = 1
train_data.loc[(train_data['Fare'] > 14.454) & (train_data['Fare'] <= 31), 'Fare']   = 2
train_data.loc[ train_data['Fare'] > 31, 'Fare'] = 3
train_data['Fare'] = train_data['Fare'].astype(int)
    
test_data.loc[test_data['Fare']<=7.91,'Fare']=0
test_data.loc[(test_data['Fare'] > 7.91) & (test_data['Fare'] <= 14.454), 'Fare'] = 1
test_data.loc[(test_data['Fare'] > 14.454) & (test_data['Fare'] <= 31), 'Fare']   = 2
test_data.loc[ test_data['Fare'] > 31, 'Fare'] = 3
test_data['Fare'] = test_data['Fare'].astype(int)
    

In [57]:
train_data.loc[ train_data['Age'] <= 16, 'Age'] 					       = 0
train_data.loc[(train_data['Age'] > 16) & (train_data['Age'] <= 32), 'Age'] = 1
train_data.loc[(train_data['Age'] > 32) & (train_data['Age'] <= 48), 'Age'] = 2
train_data.loc[(train_data['Age'] > 48) & (train_data['Age'] <= 64), 'Age'] = 3
train_data.loc[ train_data['Age'] > 64, 'Age'] =4;

test_data.loc[ test_data['Age'] <= 16, 'Age'] 					       = 0
test_data.loc[(test_data['Age'] > 16) & (test_data['Age'] <= 32), 'Age'] = 1
test_data.loc[(test_data['Age'] > 32) & (test_data['Age'] <= 48), 'Age'] = 2
test_data.loc[(test_data['Age'] > 48) & (test_data['Age'] <= 64), 'Age'] = 3
test_data.loc[ test_data['Age'] > 64, 'Age'] =4;

In [58]:
train_data['FamilySize'] = train_data['SibSp'] + train_data['Parch'] + 1
# Create new feature IsAlone from FamilySize
train_data['IsAlone'] = 0
train_data.loc[train_data['FamilySize'] == 1, 'IsAlone'] = 1

test_data['FamilySize'] = test_data['SibSp'] + test_data['Parch'] + 1
# Create new feature IsAlone from FamilySize
test_data['IsAlone'] = 0
test_data.loc[test_data['FamilySize'] == 1, 'IsAlone'] = 1

In [59]:
train_data=train_data.drop(['SibSp'],axis=1)
test_data=test_data.drop(['SibSp'],axis=1)

In [60]:
train_data=train_data.drop(['Parch'],axis=1)
test_data=test_data.drop(['Parch'],axis=1)

In [61]:
#mean=survival rate
#count=observation
#sum=survived

In [62]:
train_data[['Age','Survived']].groupby(['Age']).agg(['mean','count','sum'])

Survived           
         mean count  sum
Age                     
0.0  0.548077   104   57
1.0  0.327345   501  164
2.0  0.436893   206   90
3.0  0.434783    69   30
4.0  0.090909    11    1

In [63]:
train_data[['Fare','Survived']].groupby(['Fare']).agg(['mean','count','sum'])

Survived           
          mean count  sum
Fare                     
0     0.197309   223   44
1     0.308756   217   67
2     0.445415   229  102
3     0.581081   222  129

In [64]:
train_data[['Embarked','Survived']].groupby(['Embarked']).agg(['mean','count','sum'])

Survived           
              mean count  sum
Embarked                     
0         0.339009   646  219
1         0.553571   168   93
2         0.389610    77   30

In [65]:
train_data[['Pclass','Survived']].groupby(['Pclass']).agg(['mean','count','sum'])

Survived           
            mean count  sum
Pclass                     
1       0.629630   216  136
2       0.472826   184   87
3       0.242363   491  119

In [66]:
train_data[['IsAlone','Survived']].groupby(['IsAlone']).agg(['mean','count','sum'])

Survived           
             mean count  sum
IsAlone                     
0        0.505650   354  179
1        0.303538   537  163

In [67]:
label_Age = LabelEncoder()
Age_numerical = label_Age.fit_transform(train_data['Age'])
Age_numerical_classes = label_Age.classes_

label_Age_test = LabelEncoder()
Age_test_numerical = label_Age_test.fit_transform(test_data['Age'])
Age_test_numerical_classes = label_Age_test.classes_
print('Classes of title feature:\n{}\n{}'.format(np.arange(len(Age_numerical_classes)),Age_numerical_classes))

Classes of title feature:
[0 1 2 3 4]
[ 0.  1.  2.  3.  4.]


In [68]:
label_Fare = LabelEncoder()
Fare_numerical = label_Fare.fit_transform(train_data['Fare'])
Fare_numerical_classes = label_Fare.classes_

label_Fare_test = LabelEncoder()
Fare_test_numerical = label_Fare_test.fit_transform(test_data['Fare'])
Fare_test_numerical_classes = label_Fare_test.classes_
print('Classes of title feature:\n{}\n{}'.format(np.arange(len(Fare_numerical_classes)),Fare_numerical_classes))

Classes of title feature:
[0 1 2 3]
[0 1 2 3]


In [69]:
label_Embarked = LabelEncoder()
Embarked_numerical = label_Embarked.fit_transform(train_data['Embarked'])
Embarked_numerical_classes = label_Embarked.classes_

label_test_Embarked = LabelEncoder()
Embarked_test_numerical = label_test_Embarked.fit_transform(test_data['Embarked'])
Embarked_test_numerical_classes = label_test_Embarked.classes_
print('Classes of title feature:\n{}\n{}'.format(np.arange(len(Embarked_numerical_classes)),Embarked_numerical_classes))

Classes of title feature:
[0 1 2]
[0 1 2]


In [70]:
encoder_Age=OneHotEncoder(sparse=False)#will return array
Age_onehot = encoder_Age.fit_transform(Age_numerical.reshape(-1,1))

encoder_Age_test=OneHotEncoder(sparse=False)#will return array
Age_test_onehot = encoder_Age_test.fit_transform(Age_test_numerical.reshape(-1,1))
print(Age_onehot)
print(Age_onehot.shape)

[[ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 ..., 
 [ 0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.]]
(891, 5)


In [71]:
encoder_Fare=OneHotEncoder(sparse=False)#will return array
Fare_onehot = encoder_Fare.fit_transform(Fare_numerical.reshape(-1,1))
encoder_Fare_test=OneHotEncoder(sparse=False)#will return array
Fare_test_onehot = encoder_Fare_test.fit_transform(Fare_test_numerical.reshape(-1,1))
print(Fare_onehot)
print(Fare_onehot.shape)

[[ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  1.  0.  0.]
 ..., 
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]]
(891, 4)


In [72]:
encoder_Embarked=OneHotEncoder(sparse=False)#will return array
Embarked_onehot = encoder_Embarked.fit_transform(Embarked_numerical.reshape(-1,1))

encoder_Embarked_test=OneHotEncoder(sparse=False)#will return array
Embarked_test_onehot = encoder_Embarked_test.fit_transform(Embarked_test_numerical.reshape(-1,1))
print(Embarked_onehot)
print(Embarked_onehot.shape)

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 1.  0.  0.]
 ..., 
 [ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]
(891, 3)


In [73]:
train_data.head(5)

,Survived,Pclass,Age,Fare,Embarked,Sex_female,Sex_male,Title_Army.,Title_Dr.,Title_Master.,...,Cabs_Category_B,Cabs_Category_C,Cabs_Category_D,Cabs_Category_E,Cabs_Category_F,Cabs_Category_G,Cabs_Category_None,Cabs_Category_T,FamilySize,IsAlone
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,0,3,1.0,0,0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2,0
2,1,1,2.0,3,1,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0
3,1,3,1.0,1,0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1
4,1,1,2.0,3,0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0
5,0,3,2.0,1,0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1


In [74]:
def oneHotLabel(train_data,column,onehot_labeled,class_labels):
    new_assigns={}
    for c_id,label in enumerate(class_labels):
        print(column)
        print(label)
        print(class_labels)
#         print(onehot_labeled[:,c_id])
        print(c_id,label)
        new_assigns[column+'_'+str(label)]=onehot_labeled[:,c_id]
    train_data=train_data.assign(**new_assigns)
    return train_data

In [75]:
train_data=oneHotLabel(train_data,'Age',Age_onehot,Age_numerical_classes)#age_onehot=1 0 0 #age_numerical_class=0 1 2
train_data=train_data.drop('Age',axis=1)



Age
0.0
[ 0.  1.  2.  3.  4.]
0 0.0
Age
1.0
[ 0.  1.  2.  3.  4.]
1 1.0
Age
2.0
[ 0.  1.  2.  3.  4.]
2 2.0
Age
3.0
[ 0.  1.  2.  3.  4.]
3 3.0
Age
4.0
[ 0.  1.  2.  3.  4.]
4 4.0


In [76]:
test_data=oneHotLabel(test_data,'Age',Age_test_onehot,Age_test_numerical_classes)#age_onehot=1 0 0 #age_numerical_class=0 1 2
test_data=test_data.drop('Age',axis=1)

Age
0.0
[ 0.  1.  2.  3.  4.]
0 0.0
Age
1.0
[ 0.  1.  2.  3.  4.]
1 1.0
Age
2.0
[ 0.  1.  2.  3.  4.]
2 2.0
Age
3.0
[ 0.  1.  2.  3.  4.]
3 3.0
Age
4.0
[ 0.  1.  2.  3.  4.]
4 4.0


In [77]:
train_data.head(0)

,Survived,Pclass,Fare,Embarked,Sex_female,Sex_male,Title_Army.,Title_Dr.,Title_Master.,Title_Miss.,...,Cabs_Category_G,Cabs_Category_None,Cabs_Category_T,FamilySize,IsAlone,Age_0.0,Age_1.0,Age_2.0,Age_3.0,Age_4.0
PassengerId,,,,,,,,,,,,,,,,,,,,,


In [78]:
train_data=oneHotLabel(train_data,'Fare',Fare_onehot,Fare_numerical_classes)#age_onehot=1 0 0 #age_numerical_class=0 1 2
train_data=train_data.drop('Fare',axis=1)

test_data=oneHotLabel(test_data,'Fare',Fare_test_onehot,Fare_test_numerical_classes)#age_onehot=1 0 0 #age_numerical_class=0 1 2
test_data=test_data.drop('Fare',axis=1)

Fare
0
[0 1 2 3]
0 0
Fare
1
[0 1 2 3]
1 1
Fare
2
[0 1 2 3]
2 2
Fare
3
[0 1 2 3]
3 3
Fare
0
[0 1 2 3]
0 0
Fare
1
[0 1 2 3]
1 1
Fare
2
[0 1 2 3]
2 2
Fare
3
[0 1 2 3]
3 3


In [79]:
train_data=oneHotLabel(train_data,'Embarked',Embarked_onehot,Embarked_numerical_classes)#age_onehot=1 0 0 #age_numerical_class=0 1 2
train_data=train_data.drop('Embarked',axis=1)

test_data=oneHotLabel(test_data,'Embarked',Embarked_test_onehot,Embarked_test_numerical_classes)#age_onehot=1 0 0 #age_numerical_class=0 1 2
test_data=test_data.drop('Embarked',axis=1)

Embarked
0
[0 1 2]
0 0
Embarked
1
[0 1 2]
1 1
Embarked
2
[0 1 2]
2 2
Embarked
0
[0 1 2]
0 0
Embarked
1
[0 1 2]
1 1
Embarked
2
[0 1 2]
2 2


In [80]:
train_data_columns=train_data.drop('Survived',axis=1)
pred_data=train_data['Survived']

In [81]:
train_data=train_data_columns.drop(['Title_Noble.','Cabs_Category_T'],axis=1)

test_data=test_data.drop(['Title_Dona.'],axis=1)

In [82]:

X=train_data
Y=pred_data
clf=DecisionTreeClassifier()
clf.fit(X,Y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [83]:
predictions1=clf.predict(test_data)

In [84]:
clf.score(X,Y)

0.90909090909090906

In [85]:
def kgl_submit_csv(df,predictions,savename="titanic.csv",save=False):#default [parameter in last]
    print("Number of predictions:",len(predictions))
    print("Number of passengers",len(df.index.values))
    kgl_df=pd.DataFrame({
        "PassengerId":df.index.values,
        "Survived":predictions
    })
    print(save)
    if save:
        kgl_df.to_csv(savename,index=False)
    return kgl_df

In [86]:
a=kgl_submit_csv(test_data,predictions1,save=True)

Number of predictions: 418
Number of passengers 418
True
